In [18]:
import streamlit as st
import pandas as pd
import numpy as np
import geopandas as gpd  # For handling GeoJSON data
import folium
import branca.colormap as cm
from streamlit_folium import folium_static  # Import to render folium maps in Streamlit
from sklearn.metrics import accuracy_score
pd.set_option('display.max_columns', None)


In [5]:
fico_threshold = 650
energy_score_threshold = 0.5

solstice_territory_name = "Eversource - Western MA"


def get_solstice_territory_geojson(solstice_territory_name):
    load_name = "filtered_geojsons/" + solstice_territory_name + '.geojson'
    temp = gpd.read_file(load_name)
    temp['Utility'] = solstice_territory_name
    return temp

zip_geojson = get_solstice_territory_geojson(solstice_territory_name)


person_data = pd.read_csv('data.csv', dtype={'ZIP': str})

# Ensure ZIP codes have leading zeros and handle floats
person_data['ZIP'] = person_data['ZIP'].apply(
    lambda x: str(int(float(x))).zfill(5) if pd.notnull(x) else '')

# Ensure GeoJSON ZIP codes are formatted as strings with leading zeros
zip_geojson['ZIP'] = zip_geojson['ZCTA5CE10'].astype(str).str.zfill(5)


In [6]:
solstice_territory_name

'Eversource - Western MA'

In [7]:
person_data[person_data.ZIP == '01238'].head()

,PROFILE_ID,NUMBER_OF_TRADELINES,WEIGHTED_ENERGYSCORE,WEIGHTED_ACTUAL_OUTPUT,FICO_V9_SCORE,ZIP,EQUIFAX_TYPE_CODE
20131,20347,4,0.154410,0,733,01238,FZ
47884,48422,9,0.000058,0,850,01238,DC
175156,178308,6,0.220575,0,525,01238,FA


In [208]:
import plotly.express as px

# Sample data
top_10_zip_codes = pd.DataFrame({
    'ZIP': ['00601', '00602', '00603', '00604', '00605', '00606', '00607', '00608', '00609', '00610'],
    'Qualification Increase': [50, 60, 70, 80, 60, 50, 40, 30, 20, 10]
})

# Ensure the ZIP column is treated as a string
top_10_zip_codes['ZIP'] = top_10_zip_codes['ZIP'].astype(str)

# Bar plot for top 10 ZIP codes
fig = px.bar(top_10_zip_codes, x='ZIP', y='Qualification Increase', title="Top 10 ZIP Codes by Qualification Increase")
fig.update_xaxes(type='category', tickmode='array', tickvals=top_10_zip_codes['ZIP'], ticktext=top_10_zip_codes['ZIP'])
st.plotly_chart(fig)

650

In [226]:
temp = calculate_zip_to_util(solstice_territory_name)

temp[temp.ZIP == '01238'].head()

/var/folders/4_/36zlcy3x3m12nwb2nfhmkkjc0000gp/T/ipykernel_51815/1966365357.py:59: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  zip_metrics = stats_data_person.groupby('ZIP').apply(calc_metrics)


,ZIP,Total Population,Percent Below FICO,Percent Above FICO,FICO Accuracy,EnergyScore Accuracy,Increase in Accuracy,Qualification Increase,ZCTA5CE10,GEOID10,CLASSFP10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,geometry,Utility
62,01238,3.0,0.666667,0.333333,0.666667,1.0,50.0,66.666667,01238,01238,B5,G6350,S,90220995.0,2706102.0,+42.2975014,-073.2301483,"MULTIPOLYGON (((-73.31536 42.32753, -73.3153 4...",Eversource - Western MA


In [13]:
zip_geojson = get_solstice_territory_geojson(solstice_territory_name)

person_data = pd.read_csv('data/combined_rf_stats_person.csv')

person_data['ZIP'] = person_data['ZIP'].apply(
    lambda x: str(int(float(x))).zfill(5) if pd.notnull(x) else '')
zip_geojson['ZIP'] = zip_geojson['ZCTA5CE10'].astype(str).str.zfill(5)

/var/folders/4_/36zlcy3x3m12nwb2nfhmkkjc0000gp/T/ipykernel_15170/1515440752.py:3: DtypeWarning: Columns (5,6,7,11) have mixed types. Specify dtype option on import or set low_memory=False.
  person_data = pd.read_csv('data/combined_rf_stats_person.csv')


In [31]:
solstice_territory_name = 'Ameren Illinois'

In [32]:

def calculate_zip_metrics(stats_data_person, fico_threshold, energy_score_threshold):
    stats_data_person['FICO_PASS'] = stats_data_person['FICO_V9_SCORE'] < fico_threshold
    stats_data_person['ENERGYSCORE_PASS'] = stats_data_person['WEIGHTED_ENERGYSCORE'] > energy_score_threshold

    total_population = len(stats_data_person)

    total_below_fico = stats_data_person[stats_data_person['FICO_PASS'] == False]

    total_es_accuracy = accuracy_score(stats_data_person['WEIGHTED_ACTUAL_OUTPUT'], stats_data_person['ENERGYSCORE_PASS'])
    total_fico_accuracy = accuracy_score(stats_data_person['WEIGHTED_ACTUAL_OUTPUT'], stats_data_person['FICO_PASS'])

    # how many people below the fico threshold would have been approved by energyscore
    below_fico_pass = total_below_fico[total_below_fico['WEIGHTED_ENERGYSCORE'] <= energy_score_threshold]
    below_fico_fail = total_below_fico[total_below_fico['WEIGHTED_ENERGYSCORE'] > energy_score_threshold]

    below_fico_pass_count = len(below_fico_pass)


    # what is the accuracy of this marginal approval using EnergyScore

    below_fico_es_accuracy_control = accuracy_score(total_below_fico['WEIGHTED_ACTUAL_OUTPUT'], total_below_fico['ENERGYSCORE_PASS'])

    percent_increase_in_qualifications_total = (below_fico_pass_count / total_population) * 100

    def calc_metrics(group):
        total_population = len(group)
        if total_population == 0:
            return pd.Series({
                'Total Population': 0,
                'Percent Below FICO': 0,
                'Percent Above FICO': 0,
                'FICO Accuracy': np.nan,
                'EnergyScore Accuracy': np.nan,
                'Qualification Increase': 0,
            })

        below_fico = group[group['FICO_PASS'] == False]
        below_fico['ENERGYSCORE_PREDICTION'] = below_fico['ENERGYSCORE_PASS'] == (below_fico['WEIGHTED_ACTUAL_OUTPUT'] == 0)

        above_fico = group[group['FICO_PASS'] == True]

        below_fico_pass = below_fico[below_fico['WEIGHTED_ENERGYSCORE'] <= energy_score_threshold]
        pct_below_fico = len(below_fico) / total_population
        pct_above_fico = len(above_fico) / total_population
        percent_increase_in_qualifications = (len(below_fico_pass) / total_population) * 100 if len(below_fico_pass) > 0 else 0

        group['FICO_PREDICTION'] = group['FICO_PASS'] == (group['WEIGHTED_ACTUAL_OUTPUT'] == 0)
        fico_accuracy = accuracy_score(group['WEIGHTED_ACTUAL_OUTPUT'], group['FICO_PREDICTION'])

        group['ENERGYSCORE_PREDICTION'] = group['ENERGYSCORE_PASS'] == (group['WEIGHTED_ACTUAL_OUTPUT'] == 0)
        energy_accuracy = accuracy_score(group['WEIGHTED_ACTUAL_OUTPUT'], group['ENERGYSCORE_PREDICTION'])

        below_fico_es_accuracy = accuracy_score(below_fico['WEIGHTED_ACTUAL_OUTPUT'], below_fico['ENERGYSCORE_PREDICTION'])

        avg_fico = group['FICO_V9_SCORE'].mean()
        avg_es = group['WEIGHTED_ENERGYSCORE'].mean()

        if fico_accuracy == 0:
            pct_increase_accuracy_es = 0
        else:
            pct_increase_accuracy_es = (energy_accuracy - fico_accuracy) / fico_accuracy * 100

        return pd.Series({
            'Total Population': total_population,
            'Percent Below FICO': pct_below_fico,
            'Percent Above FICO': pct_above_fico,
            'FICO Accuracy': fico_accuracy,
            'Total EnergyScore Accuracy': energy_accuracy,
            'Sub-FICO EnergyScore Accuracy': below_fico_es_accuracy,
            'Increase in Accuracy': pct_increase_accuracy_es,
            'Qualification Increase': percent_increase_in_qualifications,
            'Average FICO': avg_fico,
            'Average EnergyScore': avg_es,
            'Control ES Accuracy': total_es_accuracy, 
            'Control FICO Accuracy' : total_fico_accuracy,
            'Control Marginal Accuracy': (total_es_accuracy - total_fico_accuracy) / total_fico_accuracy * 100,
            'below_fico_pass_count': below_fico_pass_count, 
            'below_fico_es_accuracy': below_fico_es_accuracy_control, 
            'percent_increase_in_qualifications': percent_increase_in_qualifications_total

        })

    zip_metrics = stats_data_person.groupby('ZIP').apply(calc_metrics)
    zip_metrics = zip_metrics.reset_index()
    return zip_metrics

def calculate_zip_to_util(solstice_territory_name):
    state_util = get_solstice_territory_geojson(solstice_territory_name)
    zip_metrics = calculate_zip_metrics(person_data, fico_threshold, energy_score_threshold)
    zip_level_geo = pd.merge(zip_metrics, state_util, on='ZIP', how='left')
    zip_level_geo = zip_level_geo.dropna(subset=['geometry'])
    zip_level_geo = gpd.GeoDataFrame(zip_level_geo, crs="EPSG:4326", geometry=zip_level_geo['geometry'])
    return zip_level_geo

zip_level_geo = calculate_zip_to_util(solstice_territory_name)

/Users/jakeford/miniconda3/envs/es_env/lib/python3.11/site-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/Users/jakeford/miniconda3/envs/es_env/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/jakeford/miniconda3/envs/es_env/lib/python3.11/site-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/Users/jakeford/miniconda3/envs/es_env/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/jakeford/miniconda3/envs/es_env/lib/python3.11/site-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/Users/jakeford/miniconda3/envs/es_env/lib/python3.11/site-packages/numpy/core/_methods.py:129: R

In [34]:
zip_level_geo.head()

,ZIP,Total Population,Percent Below FICO,Percent Above FICO,FICO Accuracy,Total EnergyScore Accuracy,Sub-FICO EnergyScore Accuracy,Increase in Accuracy,Qualification Increase,Average FICO,Average EnergyScore,Control ES Accuracy,Control FICO Accuracy,Control Marginal Accuracy,below_fico_pass_count,below_fico_es_accuracy,percent_increase_in_qualifications,ZCTA5CE10,GEOID10,CLASSFP10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,geometry,Utility
18634,60537,1.0,0.0,1.0,0.0,1.0,NaN,0.0,0.0,644.0,0.007103,0.981662,0.720151,36.313289,587360.0,0.994473,66.817283,60537,60537,B5,G6350,S,1701978.0,153777.0,+41.5604766,-088.6041028,"MULTIPOLYGON (((-88.61169 41.56529, -88.61165 ...",Ameren Illinois
18729,60911,5.0,0.4,0.6,0.4,0.8,1.0,100.0,40.0,644.4,0.216040,0.981662,0.720151,36.313289,587360.0,0.994473,66.817283,60911,60911,B5,G6350,S,166232473.0,153651.0,+40.8802426,-087.9765839,"MULTIPOLYGON (((-88.12951 40.89825, -88.12887 ...",Ameren Illinois
18735,60918,1.0,0.0,1.0,0.0,1.0,NaN,0.0,0.0,520.0,0.005482,0.981662,0.720151,36.313289,587360.0,0.994473,66.817283,60918,60918,B5,G6350,S,131349119.0,130177.0,+40.5949693,-088.0256549,"MULTIPOLYGON (((-88.11973 40.61892, -88.11945 ...",Ameren Illinois
18738,60926,1.0,1.0,0.0,1.0,1.0,1.0,0.0,100.0,781.0,0.000055,0.981662,0.720151,36.313289,587360.0,0.994473,66.817283,60926,60926,B5,G6350,S,7159743.0,0.0,+40.5663505,-087.8236568,"MULTIPOLYGON (((-87.83668 40.55919, -87.83665 ...",Ameren Illinois
18739,60927,5.0,0.8,0.2,0.8,0.8,1.0,0.0,80.0,710.4,0.190646,0.981662,0.720151,36.313289,587360.0,0.994473,66.817283,60927,60927,B5,G6350,S,190176324.0,608041.0,+40.9384894,-087.9727764,"MULTIPOLYGON (((-88.13105 40.96848, -88.13033 ...",Ameren Illinois


In [37]:
zip_level_geo['percent_increase_in_qualifications'].mean()

66.8172831248137

In [230]:
import plotly.express as px


In [225]:

def calculate_zip_metrics(stats_data_person, fico_threshold, energy_score_threshold):
    # Define pass/fail criteria based on FICO and EnergyScore
    stats_data_person['FICO_PASS'] = stats_data_person['FICO_V9_SCORE'] < fico_threshold
    stats_data_person['ENERGYSCORE_PASS'] = stats_data_person['WEIGHTED_ENERGYSCORE'] > energy_score_threshold

    def calc_metrics(group):
        total_population = len(group)
        if total_population == 0:
            return pd.Series({
                'Total Population': 0,
                'Percent Below FICO': 0,
                'Percent Above FICO': 0,
                'FICO Accuracy': np.nan,
                'EnergyScore Accuracy': np.nan,
                'Qualification Increase': 0,
            })

        # Separate below and above FICO threshold
        below_fico = group[group['FICO_PASS'] == False]
        above_fico = group[group['FICO_PASS'] == True]

        # Qualification Increase calculation
        below_fico_pass = below_fico[below_fico['WEIGHTED_ENERGYSCORE'] <= energy_score_threshold]
        pct_below_fico = len(below_fico) / total_population
        pct_above_fico = len(above_fico) / total_population
        percent_increase_in_qualifications = (len(below_fico_pass) / total_population) * 100 if len(below_fico_pass) > 0 else 0

        # Adjusted FICO accuracy calculation
        # FICO_PASS = True -> Predict not late (expect WEIGHTED_ACTUAL_OUTPUT = 0)
        # FICO_PASS = False -> Predict late (expect WEIGHTED_ACTUAL_OUTPUT = 1)
        group['FICO_PREDICTION'] = group['FICO_PASS'] == (group['WEIGHTED_ACTUAL_OUTPUT'] == 0)
        fico_accuracy = accuracy_score(group['WEIGHTED_ACTUAL_OUTPUT'], group['FICO_PREDICTION'])

        # Adjusted EnergyScore accuracy calculation
        group['ENERGYSCORE_PREDICTION'] = group['ENERGYSCORE_PASS'] == (group['WEIGHTED_ACTUAL_OUTPUT'] == 0)
        energy_accuracy = accuracy_score(group['WEIGHTED_ACTUAL_OUTPUT'], group['ENERGYSCORE_PREDICTION'])

        
        if fico_accuracy == 0:
            pct_increase_accuracy_es = 0
        else:
            pct_increase_accuracy_es = (energy_accuracy-fico_accuracy) / fico_accuracy * 100



        return pd.Series({
            'Total Population': total_population,
            'Percent Below FICO': pct_below_fico,
            'Percent Above FICO': pct_above_fico,
            'FICO Accuracy': fico_accuracy,
            'EnergyScore Accuracy': energy_accuracy,
            'Increase in Accuracy': pct_increase_accuracy_es,
            'Qualification Increase': percent_increase_in_qualifications,
        })

    # Group by ZIP and apply metrics calculation
    zip_metrics = stats_data_person.groupby('ZIP').apply(calc_metrics)
    zip_metrics = zip_metrics.reset_index()
    return zip_metrics


In [189]:


# convert to geopandas
zip_level_geo = gpd.GeoDataFrame(zip_level_geo, crs="EPSG:4326", geometry=zip_level_geo['geometry'])

In [191]:
zip_level_geo.head()

,ZIP,Total Population,Percent Below FICO,Percent Above FICO,FICO Accuracy,EnergyScore Accuracy,Qualification Increase,ZCTA5CE10,GEOID10,CLASSFP10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,geometry,Utility
1979,10512,9.0,0.00,1.00,NaN,NaN,0.0,10512,10512,B5,G6350,S,141331607.0,14842472.0,+41.4576193,-073.7246077,"MULTIPOLYGON (((-73.82153 41.46554, -73.82151 ...",Central Hudson
1981,10516,2.0,0.00,1.00,NaN,NaN,0.0,10516,10516,B5,G6350,S,81626886.0,1587119.0,+41.4619726,-073.8749131,"MULTIPOLYGON (((-73.97859 41.44212, -73.9783 4...",Central Hudson
1987,10524,3.0,0.00,1.00,NaN,NaN,0.0,10524,10524,B5,G6350,S,53839856.0,839861.0,+41.3753251,-073.9262166,"MULTIPOLYGON (((-73.9795 41.32216, -73.97924 4...",Central Hudson
2014,10579,4.0,0.25,0.75,1.0,1.0,25.0,10579,10579,B5,G6350,S,84788614.0,3613306.0,+41.3953643,-073.8390532,"MULTIPOLYGON (((-73.89245 41.34375, -73.89193 ...",Central Hudson
2049,10916,4.0,0.25,0.75,1.0,1.0,25.0,10916,10916,B5,G6350,S,52319522.0,234779.0,+41.4423435,-074.2505767,"MULTIPOLYGON (((-74.32285 41.43885, -74.32243 ...",Central Hudson


In [195]:
zip_level_geo['FICO Accuracy'].describe()

count    50.000000
mean      0.987143
std       0.073143
min       0.500000
25%       1.000000
50%       1.000000
75%       1.000000
max       1.000000
Name: FICO Accuracy, dtype: float64

In [ ]:
state_util = get_solstice_territory_geojson(solstice_territory_name)

state_util.head()

In [ ]:
zip_metrics = calculate_zip_metrics(
    person_data, fico_threshold, energy_score_threshold)

zip_metrics.head()

In [ ]:
zip_level_geo = pd.merge(zip_metrics, state_util, on='ZIP', how='left')
zip_level_geo = zip_level_geo.dropna(subset=['geometry'])
zip_level_geo.head()

# zip_level_geo = gpd.GeoDataFrame(zip_level_geo, geometry='geometry')

# zip_level_geo = zip_level_geo.to_crs(state_util.crs)

# zip_level_geo.head()

# zip_level_geo.Utility.value_counts()

print(zip_level_geo.shape)

zip_level_geo.head()

In [ ]:
zip_metrics = calculate_zip_metrics(
    person_data, fico_threshold, energy_score_threshold)
zip_level_geo = pd.merge(zip_metrics, zip_geojson, on='ZIP', how='left')
zip_level_geo = zip_level_geo.dropna(subset=['geometry'])
zip_level_geo = gpd.GeoDataFrame(zip_level_geo, geometry='geometry')

zip_level_geo = zip_level_geo.to_crs(state_util.crs)

zip_level_geo['geometry'] = zip_level_geo.representative_point()

zip_level_geo = gpd.sjoin(
    zip_level_geo, state_util, how='left', predicate='within')


# Use the new function to calculate ZIP to utility metrics
state_util = calculate_zip_to_util(zip_level_geo, solstice_territory_name)

In [ ]:
zip_level_geo.head()

In [ ]:
# test to combine the output files of Equifax and Experian person level data

eq_output = pd.read_csv('../energyscore-model/data/standard/equifax_rf_stats_person.csv')
eq_output['ZIP'] = eq_output['ZIP'].apply(lambda x: str(int(float(x))).zfill(5) if pd.notnull(x) else '')
eq_output.head()

print(eq_output.shape)

In [4]:
ex_output = pd.read_csv('../energyscore-model/data/standard/combined_rf_stats_person.csv')
ex_output['ZIP'] = ex_output['ZIP'].apply(lambda x: str(int(float(x))).zfill(5) if pd.notnull(x) else '')

ex_output.head()

# print(eq_output.shape)


/var/folders/4_/36zlcy3x3m12nwb2nfhmkkjc0000gp/T/ipykernel_15170/3922379719.py:1: DtypeWarning: Columns (5,6,7,11) have mixed types. Specify dtype option on import or set low_memory=False.
  ex_output = pd.read_csv('../energyscore-model/data/standard/combined_rf_stats_person.csv')


,PROFILE_ID,NUMBER_OF_TRADELINES,WEIGHTED_ENERGYSCORE,WEIGHTED_ACTUAL_OUTPUT,FICO_V9_SCORE,EXPERIAN_TYPE_CODE,HOMEOWNER_IN,RENTER_IN,INCOME_INSIGHT_W2,ZIP,bucket_on_time,EQUIFAX_TYPE_CODE
0,388,1,0.000045,0,637.0,18,U,U,81.0,90802,0.01,NaN
1,769,1,0.000924,0,552.0,07,U,U,46.0,60137,0.01,NaN
2,1046,3,0.000224,0,604.0,0G,Y,U,46.0,07052,0.01,NaN
3,1096,1,0.012244,0,750.0,00,Y,U,84.0,85043,0.02,NaN
4,1233,1,0.001232,0,603.0,07,Y,U,34.0,01075,0.01,NaN


In [119]:
combined_columns = ['PROFILE_ID', 'NUMBER_OF_TRADELINES', 'WEIGHTED_ENERGYSCORE', 
                    'WEIGHTED_ACTUAL_OUTPUT', 'FICO_V9_SCORE', 'ZIP']


eq_output = eq_output[combined_columns]
ex_output = ex_output[combined_columns]

combined_df = pd.concat([ex_output, eq_output], ignore_index = True)
# combined_output = combined_output[combined_columns]
# combined_output.head()

In [ ]:
combined_df.shape

In [ ]:
combined_df.head()

In [2]:
# Load the GeoJSON file
zip_geojson = gpd.read_file('demo_zips.geojson')

# Load person data, forcing ZIP to be read as strings
person_data = pd.read_csv('data.csv', dtype={'ZIP': str})

# Ensure ZIP codes have leading zeros and handle floats
person_data['ZIP'] = person_data['ZIP'].apply(lambda x: str(int(float(x))).zfill(5) if pd.notnull(x) else '')

# Ensure GeoJSON ZIP codes are formatted as strings with leading zeros
zip_geojson['ZIP'] = zip_geojson['ZCTA5CE10'].astype(str).str.zfill(5)


In [20]:
energy_score_threshold=0.5

In [ ]:
person_data.shape

In [ ]:
person_data.head()

In [ ]:
person_data[person_data['ZIP']=='01001'].head(15)

In [ ]:
7/11

In [ ]:
zip_metrics.head()

In [60]:
# Function to calculate metrics for each ZIP
def calculate_zip_metrics(stats_data_person, fico_threshold, energy_score_threshold):
    stats_data_person['FICO_PASS'] = stats_data_person['FICO_V9_SCORE'] < fico_threshold
    stats_data_person['ENERGYSCORE_PASS'] = stats_data_person['WEIGHTED_ENERGYSCORE'] > energy_score_threshold

    def calc_metrics(group):
        total_population = len(group)

        if total_population == 0:
            return pd.Series({
                'Total Population': 0,
                'Percent Below FICO': 0,
                'Percent Above FICO': 0,
                'FICO Accuracy': np.nan,
                'EnergyScore Accuracy': np.nan,
                'Qualification Increase': 0,
            })
        
        above_fico = group[group['FICO_V9_SCORE'] > fico_threshold]
        below_fico = group[group['FICO_V9_SCORE'] < fico_threshold]

        below_fico_pass = below_fico[below_fico['WEIGHTED_ENERGYSCORE'] <= energy_score_threshold]
        
        pct_below_fico = len(below_fico) / total_population
        pct_above_fico = len(above_fico) / total_population

        percent_increase_in_qualifications = (len(below_fico_pass) / total_population) * 100 if len(below_fico_pass) > 0 else 0
        numeric_increase_in_qualifications = len(below_fico_pass) if len(below_fico_pass) > 0 else 0

        energy_accuracy = accuracy_score(group['WEIGHTED_ACTUAL_OUTPUT'], group['ENERGYSCORE_PASS']) #if len(below_fico) > 0 else np.nan

        fico_accuracy = accuracy_score(group['WEIGHTED_ACTUAL_OUTPUT'], group['FICO_PASS']) #if len(below_fico) > 0 else np.nan

        return pd.Series({
            'Total Population': total_population,
            'Percent Below FICO': pct_below_fico,
            'Percent Above FICO': pct_above_fico,
            'FICO Accuracy': fico_accuracy,
            'EnergyScore Accuracy': energy_accuracy,
            'Qualification Increase': percent_increase_in_qualifications,
            'Numeric Increase': numeric_increase_in_qualifications,
        })

    # Group by ZIP and apply metrics calculation
    zip_metrics = stats_data_person.groupby('ZIP').apply(calc_metrics)
    zip_metrics = zip_metrics.reset_index()
    return zip_metrics


# Function to calculate ZIP to utility mapping and display on the map
def calculate_zip_to_util(zip_level_geo, state_name):
    # Load the utility data for the state
    state_util = load_state_util(state_name)
    state_util.rename(columns={'new_name': 'Utility'}, inplace=True)

    # Ensure ZIP code geometries have the same projection as the utility data
    zip_level_geo = zip_level_geo.to_crs(state_util.crs)

    # Convert the ZIP geometries to representative points
    zip_level_geo['geometry'] = zip_level_geo.representative_point()

    # Perform spatial join with utility data based on point locations
    zip_level_geo = gpd.sjoin(zip_level_geo, state_util, how='left', predicate='within')

    # Group by utility name ('new_name') and calculate the mean of 'Qualification Increase'
    zip_to_util = zip_level_geo.groupby('Utility')['Qualification Increase'].mean().reset_index()

    # Merge utility data with the calculated qualification increase
    state_util = state_util.merge(zip_to_util, on='Utility', how='left')

    return state_util


In [67]:
state_name = "Massachusetts"
fico_threshold = 650
energy_score_threshold = .5


In [ ]:
zip_metrics = calculate_zip_metrics(person_data, fico_threshold, energy_score_threshold)
zip_level_geo = pd.merge(zip_metrics, zip_geojson, on='ZIP', how='left')
zip_level_geo = zip_level_geo.dropna(subset=['geometry'])
zip_level_geo = gpd.GeoDataFrame(zip_level_geo, geometry='geometry')

print(zip_metrics['EnergyScore Accuracy'].describe())

print(zip_metrics['FICO Accuracy'].describe())


In [ ]:
zip_metrics.head()

In [ ]:
zip_metrics['EnergyScore Accuracy'].describe()

In [ ]:
zip_metrics['FICO Accuracy'].describe()

In [ ]:
zip_level_geo.head()

In [ ]:
import geopandas as gpd
import pandas as pd
import json
import os

# Step 1: Load the GeoJSON file containing all US ZIP codes
us_zip_geojson = gpd.read_file('/Users/jakeford/solstice/energyscore-model/us_zips.geojson')  # Adjust the file path as needed
us_zip_geojson['ZIP'] = us_zip_geojson['ZCTA5CE10'].astype(str).str.zfill(5)

# Step 2: Load the CSV file containing filter conditions for ZIP codes
df = pd.read_csv('//Users/jakeford/Downloads/82284_2024_10_16.csv')  # Load your CSV file

# Create a directory to save the output GeoJSON files
output_dir = 'filtered_geojsons'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)


In [ ]:
df.head()

In [83]:
def extract_zipcodes(json_string):
    try:
        # Replace single quotes with double quotes to make it valid JSON
        json_string = json_string.replace("'", "\"")
        # Load the string as a dictionary and extract the list
        zipcodes_list = json.loads(json_string)['list']
        return zipcodes_list
    except (json.JSONDecodeError, KeyError) as e:
        print(f"Error decoding JSON: {e}")
        return []

# Apply the function to the 'zipcodes' column
df['zipcodes_list'] = df['zipcodes'].apply(extract_zipcodes)


In [ ]:
df

In [ ]:
for index, row in df.iterrows():
    # Get the list of zip codes for the current row
    zipcodes_list = row['zipcodes_list']  # Assuming this column now contains the Python list of zip codes
    
    # Filter the GeoJSON data to include only the relevant ZIP codes
    filtered_zip_geojson = us_zip_geojson[us_zip_geojson['ZIP'].isin(zipcodes_list)]
    
    # Use the 'friendly_name' column for the filename
    friendly_name = row['friendly_name']
    output_filename = os.path.join(output_dir, f"{friendly_name}.geojson")
    
    # Save the filtered GeoJSON file
    filtered_zip_geojson.to_file(output_filename, driver='GeoJSON')
    
    print(f"Saved filtered GeoJSON for {friendly_name} to {output_filename}")

In [ ]:
fico_threshold = 650
energy_score_threshold = 0.5

In [ ]:
zip_geojson = gpd.read_file('demo_zips.geojson')

In [ ]:
df

In [ ]:
zip_level_geo.shape

In [ ]:
zip_level_geo['Total Population'].describe()

In [7]:
import fiona

In [ ]:
# Load the GeoJSON file
#zip_geojson = gpd.read_file('select_zips.geojson')

zip_geojson = gpd.read_file('demo_zips.geojson')

# Load person data, forcing ZIP to be read as strings
person_data = pd.read_csv('data.csv', dtype={'ZIP': str})

# Ensure ZIP codes have leading zeros and handle floats
person_data['ZIP'] = person_data['ZIP'].apply(lambda x: str(int(float(x))).zfill(5) if pd.notnull(x) else '')

# Ensure GeoJSON ZIP codes are formatted as strings with leading zeros
zip_geojson['ZIP'] = zip_geojson['ZCTA5CE10'].astype(str).str.zfill(5)

# Check if ZIPs were properly converted
print(person_data['ZIP'].head())
print(zip_geojson['ZIP'].head())


In [ ]:
# Function to calculate metrics for each ZIP
def calculate_zip_metrics(stats_data_person, fico_cutoff, energyscore_cutoff):
    # Create masks for conditions
    stats_data_person['FICO_PASS'] = stats_data_person['FICO_V9_SCORE'] > fico_cutoff
    stats_data_person['ENERGYSCORE_PASS'] = stats_data_person['WEIGHTED_ENERGYSCORE'] > energyscore_cutoff

    # Function to calculate various metrics
    def calc_metrics(group):
        total_population = len(group)
        if total_population == 0:
            return pd.Series({
                'Total Population': 0,
                'Percent Below FICO': 0,
                'Percent Above FICO': 0,
                'FICO Accuracy': np.nan,
                'EnergyScore Accuracy': np.nan,
                'Qualification Increase': 0,
            })
        
        
        below_fico = group[group['FICO_PASS'] == False]
        above_fico = group[group['FICO_PASS'] == True]

        if len(below_fico) == 0:
            return pd.Series({
                'Total Population': total_population,
                'Percent Below FICO': 0,
                'Percent Above FICO': len(above_fico) / total_population,
                'FICO Accuracy': np.nan,
                'EnergyScore Accuracy': np.nan,
                'Qualification Increase': 0,
            })

        below_fico_pass = below_fico[below_fico['WEIGHTED_ENERGYSCORE'] <= energyscore_cutoff]
        below_fico_fail = below_fico[below_fico['WEIGHTED_ENERGYSCORE'] > energyscore_cutoff]

        pct_below_fico = len(below_fico) / total_population
        pct_above_fico = len(above_fico) / total_population

        if len(below_fico_pass) == 0:
            percent_increase_in_qualifications = 0
        else:
            percent_increase_in_qualifications = (len(below_fico_pass) / total_population) * 100

        # get FICO accuracy, precision, recall, f1 and roc_auc score
       # fico_accuracy = accuracy_score(below_fico['WEIGHTED_ACTUAL_OUTPUT'], below_fico['WEIGHTED_ENERGYSCORE'] > energyscore_cutoff) if len(below_fico) > 0 else np.nan
        fico_accuracy = accuracy_score(below_fico['WEIGHTED_ACTUAL_OUTPUT'], below_fico['FICO_PASS']) if len(below_fico) > 0 else np.nan

        energy_accuracy = accuracy_score(below_fico['WEIGHTED_ACTUAL_OUTPUT'], below_fico['ENERGYSCORE_PASS']) if len(below_fico) > 0 else np.nan

      #  accuracy_increase = energy_accuracy - fico_accuracy

        return pd.Series({
            'Total Population': total_population,
            'Percent Below FICO': pct_below_fico,
            'Percent Above FICO': pct_above_fico,
            'FICO Accuracy': fico_accuracy,
            'EnergyScore Accuracy': energy_accuracy,
            'Qualification Increase': percent_increase_in_qualifications,
          #  'Accuracy Percentage Increase': accuracy_increase,

           # 'Accuracy Percentage Increase': (energy_accuracy - fico_accuracy) / fico_accuracy * 100 if fico_accuracy * energy_accuracy> 0 else 0,
           

        })

    # Group by ZIP and apply metrics calculation
    zip_metrics = stats_data_person.groupby('ZIP').apply(calc_metrics)

    zip_metrics = zip_metrics.reset_index()
    return zip_metrics

fico_threshold = 700
energy_score_threshold = 0.5
# Calculate metrics and merge with geo data
zip_metrics = calculate_zip_metrics(person_data, fico_threshold, energy_score_threshold)
zip_level_geo = pd.merge(zip_metrics, zip_geojson, on='ZIP', how='left')
zip_level_geo = zip_level_geo.dropna(subset=['geometry'])
zip_level_geo = gpd.GeoDataFrame(zip_level_geo, geometry='geometry')


In [ ]:
zip_level_geo

In [ ]:


def load_state_util(state_name):
    if state_name == 'New Mexico':
        temp = gpd.read_file('nm_utils.geojson')
        temp = temp[['new_name', 'geometry']]
        return temp
    elif state_name == 'Massachusetts':
        return gpd.read_file('ma_utils.geojson')


def calculate_zip_to_util(zip_level_geo, util_name, state_name):
    # Load the utility data for the state
    state_util = load_state_util(state_name)

    # Ensure ZIP code geometries have the same projection as the utility data
    zip_level_geo = zip_level_geo.to_crs(state_util.crs)

    # Convert the ZIP geometries to representative points (instead of centroids)
    zip_level_geo['geometry'] = zip_level_geo.representative_point()

    # Perform spatial join with utility data based on point locations using predicate
    zip_level_geo = gpd.sjoin(zip_level_geo, state_util, how='left', predicate='within')

    # Group by utility name ('new_name') and calculate the mean of 'Qualification Increase'
    zip_to_util = zip_level_geo.groupby('new_name')['Qualification Increase'].mean().reset_index()

    return zip_to_util

# Example usage
state_name = 'New Mexico'
util_name = 'Xcel'
zip_to_util = calculate_zip_to_util(zip_level_geo, util_name, state_name)
zip_to_util.head()


In [ ]:
zip_metrics

In [ ]:
zip_level_geo.shape

In [ ]:
zip_metrics['FICO Accuracy'].mean(),zip_metrics['EnergyScore Accuracy'].mean()

In [ ]:
zip_geojson.head()